In [1]:
%matplotlib widget

In [2]:
import pandas as pd
import numpy as np
import random

from lazypredict.Supervised import LazyClassifier

/home/lukasadamowicz/miniconda3/envs/pfymu/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
data = pd.read_hdf('../feature_exploration/features.h5', key='incl_stairs')

In [4]:
# get the subjects for which LOSO actually makes sense: those with multiple activities (ie more than just walking)
gbc = data.groupby(['Subject', 'Activity'], as_index=False).count()
loso_subjects = [i for i in gbc.Subject.unique() if gbc.loc[gbc.Subject == i].shape[0] > 3]

random.seed(5)  # fix the generation so that its the same every time
random.shuffle(loso_subjects)

training_masks = []
validation_masks = []
testing_masks = []

for i in range(0, len(loso_subjects), 4):
    tr_m = np.ones(data.shape[0], dtype='bool')
    v_m = np.zeros(data.shape[0], dtype='bool')
    te_m = np.zeros(data.shape[0], dtype='bool')
    
    for j in range(4):
        tr_m &= (data.Subject != loso_subjects[i+j]).values
    for j in range(2):
        v_m |= (data.Subject == loso_subjects[i+j]).values
    for j in range(2):
        te_m |= (data.Subject == loso_subjects[i+j+2]).values
    
    training_masks.append(tr_m)
    validation_masks.append(v_m)
    testing_masks.append(te_m)

masks = (training_masks, validation_masks, testing_masks)

In [5]:
X = data.iloc[:, 3:]
Y = data.Label

In [7]:
from lazypredict.Supervised import CLASSIFIERS

In [12]:
_CLASSIFIERS = CLASSIFIERS.copy()  # back up

CLASSIFIERS = CLASSIFIERS[:1]
CLASSIFIERS

[('AdaBoostClassifier', sklearn.ensemble._weight_boosting.AdaBoostClassifier)]

In [13]:
clf = LazyClassifier()

In [14]:
clf.fit(X.loc[training_masks[0]], X[validation_masks[0]], Y[training_masks[0]], Y[validation_masks[0]])

 30%|███       | 9/30 [01:17<03:01,  8.65s/it]


KeyboardInterrupt: 